# FiMMIA: scaling semantic perturbation-based membership inference across modalities

<p align="center">
  <picture>
    <img alt="FiMMIA" src="../docs/FiMMIA_system_overview.png" style="max-width: 100%;">
  </picture>
</p>

The system is the first collection of models and pipelines for membership inference attacks against multimodal large language models, built initially with a priority for the Russian language, and extendable to any other language or dataset. 
Pipeline supports different modalities: image, audio and video. In our experiments, we focus on [MERA datasets](https://github.com/MERA-Evaluation/MERA), however, the presented pipeline can be generalized to other languages. The system is a set of models and Python scripts in a GitHub repository. 

We support two major functionalities for image, audio and video modalities: inference of membership detection model and training pipeline for new datasets.

Pretrained models available on 🤗 HuggingFace [FiMMIA collection](https://huggingface.co/collections/ai-forever/fimmia).

## Installation
Download repository and install requirements.

```bash
git clone https://github.com/ai-forever/data_leakage_detect
cd data_leakage_detect
pip install -r requirements.txt
```

## Example
For example of usage of FiMMIA we take dataset [MERA-evaluation/CommonVideoQA](https://huggingface.co/datasets/MERA-evaluation/CommonVideoQA), train FiMMIA neural network model and will see inference.

### Train FiMMIA model on dataset MERA-evaluation/CommonVideoQA
We divided the training algorithm into the following subsequent steps with some modifications:
* [Data preparation](#Data-preparation)
* [SFT MLLM finetuning](#SFT-MLLM-finetuning)
* [Neighbor generation](#Neighbor-generation)
* [Embedding generation](#Embedding-generation)
* [Loss computation](#Loss-computation)
* [Training the attack model](#Training-the-attack-model)
* [Run MIA inference](#Run-MIA-inference)

#### Data preparation
Download data and convert to own working format. We should convert our dataset into pandas format with following structure:

| input | answer | video | ds_name  |
|----------|--------|-------|----------|

* `input` example:

```text
Очень бы хотелось получить решение такой задачи. В этой задаче необходимо выбрать правильный ответ из четырех предложенных вариантов на основе переданных вопроса и видео.

Имеется 1 видеофайл

Желательно, чтобы вы ознакомились с данными и решили задачу, выбрав из вариантов ответа один или несколько правильных.

Видеофайл: <video>
Вопрос:
Почему баскетболист отдал мяч игроку в майке другого цвета?

A. Он перепутал соперника с игроком своей команды.
B. Эти игроки тренируются вместе, цвет одежды не имеет значения.
C. Он решил так поступить, потому что его команда всё равно проигрывает.
D. Он специально подыгрывает другой команде.

Первому из предложенных вариантов ответа присваивается литера А, второму литера B, третьему литера C и так далее по английскому алфавиту. В качестве ответа будет правильно вывести литеру, соответствующую верному варианту ответа из предложенных. Это лучше сделать в таком формате: по завершении рассуждений пишется слово ОТВЕТ, затем через пробел выводится выбранная литера. ОТВЕТ: 
```

* `answer` example: 'B'.
* `video` - is the modality column. For audio we should put `audio`, for image - `image`.
* `ds_name` is the dataset name. For example `CommonVideoQA`.

In [2]:
from datasets import load_dataset
import numpy as np
import os
import warnings
import torch
import sys
import torchvision
from fractions import Fraction
import pandas as pd
sys.path.append("../")
warnings.filterwarnings("ignore")
def save_video(line, video_path, video_codec="libx264"):
    video_codec = "libx264"
    container = line["inputs"]["video"].container
    video_fps = Fraction(line["inputs"]["video"].get_metadata()["video"]["fps"][0])
    total_frames = container.streams.video[0].frames
    container.seek(0)
    frames = list(container.decode(video=0))
    video_array = torch.from_numpy(np.stack([x.to_ndarray(format="rgb24") for x in frames]))
    torchvision.io.write_video(
        filename=video_path,
        video_array=video_array,
        fps=video_fps,
        video_codec=video_codec
    )

In [3]:
working_dir = "data/"
ds_name = "CommonVideoQA"
ds_dir = os.path.join(working_dir, ds_name)
ds_samples_dir = os.path.join(ds_dir, "samples")
os.makedirs(ds_samples_dir, exist_ok=True)

In [ ]:
ds = load_dataset("MERA-evaluation/CommonVideoQA")

In [58]:
lines = []
for line in ds["test"]:
    idx = line["meta"]["id"]
    video_path = os.path.join(ds_dir, "samples", f"video_{idx}.mp4")
    save_video(line, video_path)
    lines.append({
        "question": line["instruction"].format(**line["inputs"]) + "ОТВЕТ: ",
        "answer": line["outputs"],
        "video": video_path,
        "ds_name": ds_name
    })

In [73]:
df_path = os.path.join(working_dir, "train_all.csv")

In [74]:
df.to_csv(df_path, index=False)

In [75]:
df = pd.read_csv(df_path)

In [76]:
df.head()

,input,answer,video,ds_name
0,Привет! Поможешь?\n\nМне попалась такая задача...,B,data/CommonVideoQA/samples/video787.mp4,CommonVideoQA
1,Очень бы хотелось получить решение такой задач...,B,data/CommonVideoQA/samples/video451.mp4,CommonVideoQA
2,Внимание!\n\nВ датасете к задаче идёт такой пр...,D,data/CommonVideoQA/samples/video912.mp4,CommonVideoQA
3,Требуется помощь.\n\nНужно сделать следующее. ...,C,data/CommonVideoQA/samples/video625.mp4,CommonVideoQA
4,Тут задача.\n\nИмеется 1 видеофайл\n\nРеши зад...,A,data/CommonVideoQA/samples/video737.mp4,CommonVideoQA


**Split data into train and test**

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)
train_df.to_csv("data/train.csv", index=False)
test_df.to_csv("data/test.csv", index=False)

---

#### SFT MLLM finetuning
For obtaining positive labels that indicates leak we finetune MLLM. Here we obtain two MLLMs: original model $\mathcal{M}$ without leak and $\mathcal{M}_{leak}$ with leak.

**Running command**

```bash
python job_launcher.py --script="fimmia.video.train_qwen25vl" \
  --train_df_path="data/train_all.csv" \
  --test_df_path="data/test.csv" \
  --num_train_epochs=5 \
  --model_id="Qwen/Qwen2.5-VL-3B-Instruct" \
  --output_dir=f"data/Qwen2.5-VL-3B-Instruct"
```

---

#### Neighbor generation
For each original data point $(t, s)$ we generate $K=24$ perturbed "neighbors" $(t^k_\prime, s^k_\prime)$. We apply four different perturbation techniques:
* Random masking and predicting the masks with [ai-forever/FRED-T5-1.7B](https://huggingface.co/ai-forever/FRED-T5-1.7B) model
* Deletion of random tokens
* Duplication
* Swapping of random tokens

to the text $t$ with each technique applied 6 times. % The modality data $s$ remains unchanged. 
Although, in our experiments we fix $s = s^k_\prime, \: \forall s \in D $, so the modality data remains unchanged, the pipeline can be modified to support neighbors from different modalities as well.

**Run command:**

```bash
python job_launcher.py --script="fimmia.neighbors" \
  --model_path="ai-forever/FRED-T5-1.7B" \
  --dataset_path="data/train.csv" \
  --max_text_len=4000
```

Here
* `model_path` - embedder model for masking neighbors generation
* `dataset_path` - path to dataset for generating neighbors
* `max_text_len` - max of text length in number of characters

After running this command we will replace origin file with new file with **neighbors** column. 

In [ ]:
df = pd.read_csv(df_path)

In [103]:
df.head()

,input,answer,video,ds_name,neighbors
0,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,['Видеофайл: <video> Вопрос: Что происходит в ...
1,Требуется помощь.\n\nНужно сделать следующее. ...,C,data/CommonVideoQA/samples/video625.mp4,CommonVideoQA,['Требуется помощь. Нужно сделать следующее. В...
2,Сформулирована задача.\n\nВ задаче требуется с...,A,data/CommonVideoQA/samples/video859.mp4,CommonVideoQA,['Сформулирована задача. _В_ задаче _предложен...
3,Внимание!\n\nВ датасете к задаче идёт такой пр...,C,data/CommonVideoQA/samples/video538.mp4,CommonVideoQA,['Внимание! В _приложении_ к _видео_ идёт тако...
4,Требуется помощь.\n\nНужно сделать следующее. ...,A,data/CommonVideoQA/samples/video835.mp4,CommonVideoQA,['Требуется помощь. Нужно сделать следующее. В...


In [ ]:
inpt = df.input[0]
answer = df.answer[0]
neighbors = eval(df.neighbors[0])
masking_neighbor = neighbors[3]
deletion_neighbor = neighbors[7]
duplication_neighbor = neighbors[13]
swapping_neighbor = neighbors[-1]

* **input** for sample is

Видеофайл: \<video\>

Вопрос:

Что происходит в видео с 00.510 по 11.460 секунды?

A. Помешивают ложкой <span style="color:blue">**разогревшееся блюдо**</span> в сковороде.

B. Помешивают <span style="color:blue">**ложкой**</span> готовое блюдо в кастрюле.

C. Разделяют <span style="color:blue">**замёрзший продукт**</span> ложкой в сковороде, помогая размораживаться.

D. Закрывают сковороду крышкой.

* **Random masking** neighbor

Видеофайл: \<video\>

Вопрос:

Что происходит в видео с 00.510 по 11.460 секунды?

A. Помешивают ложкой <span style="color:red">**ройбуш ройбуш и готовое блюдо**</span> в сковороде.

B. Помешивают <span style="color:red">**ройбуш и** </span>готовое блюдо в кастрюле.

C. Разделяют <span style="color:red">**ройбуш и ройбуш**</span> ложкой в сковороде, помогая размораживаться.

D. Закрывают сковороду крышкой.

---

* **Deletion** neighbor

Видеофайл: \<video\>

Вопрос:

Что происходит в видео с 00.510 по 11.460 секунды?

A. Помешивают ложкой ~~**разогревшееся**~~ блюдо в сковороде.

B. Помешивают ~~**ложкой**~~ готовое блюдо в кастрюле.

C. Разделяют замёрзший продукт ложкой в сковороде, ~~**помогая**~~ размораживаться.

D. Закрывают сковороду крышкой.

---

* **Duplication** neighbor

Видеофайл: \<video\>

Вопрос:

**Что** Что **происходит** происходит в видео с 00.510 по 11.460 секунды?

A. Помешивают ложкой разогревшееся **блюдо** блюдо в сковороде.

B. Помешивают **ложкой** ложкой готовое блюдо в кастрюле.

C. Разделяют **замёрзший** замёрзший продукт ложкой в сковороде, помогая размораживаться.

D. Закрывают сковороду крышкой.

---

* **Swapping** neighbor

<span style="color:blue">**блюдо**</span> \<video\>

<span style="color:green">**размораживаться.**</span>

Что происходит в видео с <span style="color:red">**B.**</span> по 11.460 секунды?

A. Помешивают ложкой разогревшееся <span style="color:blue">**Видеофайл:**</span> в сковороде.

<span style="color:red">**00.510**</span> Помешивают ложкой готовое блюдо в кастрюле.

C. Разделяют замёрзший продукт ложкой в сковороде, помогая <span style="color:green">**Вопрос:**</span>

D. Закрывают сковороду крышкой.

---

#### Embedding generation
Then for each original text $t$ and its neighbors $t^k_\prime$ we extract their text embeddings using a fixed encoder:
$$e=\mathcal{E}(t), \quad e_{k}^{\prime} = \mathcal{E}(t_k^{\prime})$$
where $\mathcal{E}$ is [intfloat/e5-mistral-7b-instruct](https://huggingface.co/intfloat/e5-mistral-7b-instruct) in our experiments. It used to be SoTA on the [MTEB](https://huggingface.co/blog/mteb) benchmark at the time of the model experiments}.

**Run command:**

```bash
python job_launcher.py --script="fimmia.embeds_text_calc" \
  --embed_model="intfloat/e5-mistral-7b-instruct" \
  --df_path="data/train.csv"
```

Here
* `embed_model` - embedder path
* `df_path` - path to dataset for generating embeddings

After running this command we will create new directory with name `data/train_all/embeds/`. Directory contains list of files `part_0.csv`, `part_1.csv` and so on depends from dataset size. Each file has two new columns:
* **neighbor_embeds** - embedding vector for neighbor
* **input_embeds** - embedding vector for origin text

Column **input_embeds** also contains not `NaN` for each first pair neighbor and input for of dataset sample. For example not `NaN` **input_embeds** for **id=0** and **id=24**.

In [114]:
df = pd.read_csv("data/train/embeds/part_0.csv")

In [117]:
df.head(2)

,input,answer,video,ds_name,neighbor,neighbor_embeds,input_embeds
0,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,Видеофайл: <video> Вопрос: Что Что происходит ...,"[0.017215704545378685, -0.010462397709488869, ...","[0.02046734280884266, -0.011482938192784786, 0..."
1,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,Видеофайл: <video> Вопрос: готовое происходит ...,"[0.01675490476191044, -0.010078263469040394, -...",NaN


---

#### Loss computation

We compute the multimodal loss for both models $\mathcal{M}$ and $\mathcal{M}_{leak}$ on both the original and neighbor data points:
$$\mathcal{L} = \mathcal{L}(\mathcal{M}, t, s), \quad \mathcal{L}_k^{\prime} = \mathcal{L}(\mathcal{M}, t^k_{\prime}, s^k_{\prime})$$
Text input $t$ is provided to each model, accompanied by the corresponding modality $s$ image, video, or audio data in its original, unchanged form. This passed to MLLM.

**Run command:**

```bash
python job_launcher.py --script="fimmia.video.loss_calc_qwen25" \
  --model_id=Qwen/Qwen2.5-VL-3B-Instruct \
  --model_name=Qwen2.5-VL-3B-Instruct \
  --label=0 \
  --df_path="data/train.csv" \
  --part_size=5000
```
Here
* `model_id` - path MLLM model
* `model_name` - name of MLLM model (using for store results)
* `label` - label of dataset `0` for no lean or `1` for leak model (0 by default)
* `df_path` - path to dataset for calculating loss
* `part_size` - lines for split dataframe into smaller frames

After running this command we will create new directory with name `data/train_all/loss/Qwen2.5-VL-3B-Instruct/leak` or `data/train_all/loss/Qwen2.5-VL-3B-Instruct/no_leak` depends from `label`. Directory contains list of files `part_0.csv`, `part_1.csv` and so on depends from dataset size. Each file has two new columns:
* **neighbor_loss** - loss for neighbor dataset sample: $text_neighbor + image$
* **input_loss** - loss for origin dataset sample: $text + image$

For training we should produce files for both $\mathcal{M}$ and $\mathcal{M}_{leak}$ models.

For finetunned sft model **run command**:

```bash
python job_launcher.py --script="fimmia.video.loss_calc_qwen25" \
  --model_id=data/Qwen2.5-VL-3B-Instruct \
  --model_name=Qwen2.5-VL-3B-Instruct \
  --label=0 \
  --df_path="data/train.csv" \
  --part_size=5000
```

In [119]:
df = pd.read_csv("data/train/loss/Qwen2.5-VL-3B-Instruct/no_leak/part_0.csv")

In [121]:
df["video"] = [os.path.join(ds_samples_dir, os.path.split(x)[-1]) for x in df["video"]]

In [123]:
df.head(5)

,input,answer,video,ds_name,neighbor,neighbor_loss,input_loss
0,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,Видеофайл: <video> Что Вопрос: происходит в ви...,0.872269,0.863068
1,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,Видеофайл: <video> Вопрос: Что происходит в ви...,0.935770,0.863068
2,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,Видеофайл: <video> Вопрос: Что в видео с 00.51...,0.862926,0.863068
3,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,Видеофайл: <video> Вопрос: происходит видео с ...,1.260783,0.863068
4,Видеофайл: <video>\nВопрос:\nЧто происходит в ...,A,data/CommonVideoQA/samples/video419.mp4,CommonVideoQA,Видеофайл: <video> Вопрос: Что происходит сков...,1.256082,0.863068


---

#### Training the attack model
The core of FiMMIA is a binary neural network classifier trained to distinguish between models that have and have not seen the data. For each neighbor $k$ we create two training examples by computing feature differences:
$$\Delta \mathcal{L} = \mathcal{L} - \mathcal{L}^k_{\prime}, \quad \Delta e = e - e^{k}_{\prime}$$

These feature vectors are paired with labels $y \in \{0, 1\}$ indicating whether the losses came from $\mathcal{M}$ (non-leaked) or $\mathcal{M}_{leak}$ (leaked). However, absolute values of these statistics may vary across datasets and models. To make the system more stable, we apply the z-score normalization technique. During the training phase, we calculate the mean $\mu$ and standard deviation $\sigma$ for the models loss differences $\Delta \mathcal{L}$ using the evaluation data. 
$$\Delta \mathcal{L}_{norm} = \frac{\Delta \mathcal{L}-\mu}{\sigma} $$.

This process yields random batch training triplets $(\Delta \mathcal{L}_{norm}, \Delta e, y)$ per original data point. The FiMMIA detector, $f_{FiMMIA}$ is trained to predict the probability $p=f_{FiMMIA}(\Delta \mathcal{L}_{norm}, \Delta e)$ that the input features originate from a model that has been trained on the target data.

##### The detailed architecture of the FiMMIA is provided below.

1. **Input Data**
* *loss\_input* A tensor fed into the *loss\_component*.
* *embedding\_input* A tensor fed into the *embedding\_component*.

2. **loss\_component**:
* A Linear layer: 1 input feature $\rightarrow$ *projection\_size* output features.
* *Dropout(0.2)* and  *ReLU* activation.
3. **embedding\_component**:
* A Linear layer: *embedding\_size* $\rightarrow$ *embedding\_size // 2*.
* *Dropout(0.2)* and  ReLU.
* A Linear layer: \texttt{embedding\_size // 2} $\rightarrow$ 512.
* *Dropout(0.2)* and  *ReLU*.
4. **Concatenation torch.hstack**:
* The outputs from the *loss\_component(projection\_size)* and the *embedding\_component(512)* are concatenated into a single vector of size *2 \* projection\_size*.
5. **attack\_encoding**:
* A series of 6 fully connected *Linear* layers with *Dropout(0.2)* and  *ReLU* activations between them: *2 \* projection\_size* $\rightarrow$ 512 $\rightarrow$ 256 $\rightarrow$ 128 $\rightarrow$ 64 $\rightarrow$ 32.
* The final linear layer: 32 $\rightarrow$ 2 (output logits for classification).
* A final  *ReLU* activation after the last layer.

7. **Output**
* The model returns the logits (size 2).
* If labels are provided, it also calculates and returns the cross-entropy loss.

After loss and embeddings calculation we should put this together to one dataset and calculate z-score statistics for training FiMMIA neural network.

---

##### Create dataset for training FiMMIA neural network

**Run command**

```bash
python job_launcher.py --script="fimmia.utils.mds_dataset" \
  --save_dir="data/train_all_mds" \
  --model_name="Qwen2.5-VL-3B-Instruct" \
  --origin_df_path="data/train.csv" \
  --shuffle=0 \
  --labels="0,1" \
  --modality_key="video" \
  --single_file=1
```

Here
* `save_dir` - path for saving merged dataset
* `model_name` - name of MLLM model (using for store results)
* `shuffle` - not shuffle data `0` or shuffle `1`
* `labels` - list of labels in dataset
* `modality_key` - modality column
* `single_file` - run on single file or batches

After running this command we get dataset in mds format with losses and embedding.

In [3]:
from fimmia.utils.data import get_mean_std, get_streaming_ds

In [ ]:
ds = get_streaming_ds("data/train_mds")

In [ ]:
for x in ds:
    break

In [11]:
x

{'answer': 'D',
 'ds_name': 'CommonVideoQA',
 'embedding_input': array([-0.00378995,  0.00234417, -0.00746693, ...,  0.00404607,
        -0.00334608,  0.00647654]),
 'hash': '-1220520432446283628',
 'input': 'Видеофайл: <video>\nВопрос:\nКаким будет наиболее вероятное следующее действие?\n\nA. Очистка огурца от пищевой пленки.\nB. Мытье огурца.\nC. Нарезка огурца.\nD. Нарезка салата.',
 'label': 0,
 'loss_input': 1.46,
 'model_name': 'Qwen2.5-VL-3B-Instruct',
 'neighbor': 'Видеофайл: <video> Вопрос: Каким будет наиболее _эффективным_ следующее действие? A. Очистка _огурца_ от щипцов _и_  пленки. B. Мытье огурца. C. Нарезка огурца. D. Нарезка салата.',
 'num_part': 0,
 'video': 'data/CommonVideoQA/samples/video035.mp4'}

**For test dataset `data/test.csv` apply all steps**:
* [Neighbor generation](#Neighbor-generation)
* [Embedding generation](#Embedding-generation)
* [Loss computation](#Loss-computation)

---

##### calculate z-score statistics

In [14]:
from fimmia.utils.utils import save_json, load_json

In [ ]:
sigmas = get_mean_std(["data/train_mds"])

In [20]:
save_json(sigmas, "data/train_sigmas.json")

In [21]:
sigmas = load_json("data/train_sigmas.json")

In [22]:
sigmas

{'CommonVideoQA_Qwen2.5-VL-3B-Instruct': {'mean': 0.009537732455924316,
  'std': 0.15681258803478634}}

---

##### Training FiMMIA neural network

After data preparation run training of an attack model neural network FiMMIA:

```bash
python job_launcher.py --script="fimmia.train" \
  --train_dataset_path="data/train_mds" \
  --model_name="FiMMIABaseLineModelLossNormSTDV2" \
  --output_dir="data/FiMMIA-Video" \
  --num_train_epochs=10 \
  --optim="adafactor" \
  --learning_rate=0.00005 \
  --max_grad_norm=10 \
  --warmup_ratio=0.03 \
  --sigmas_path="data/train_sigmas.json" \
  --sigmas_type="std"
```

Here
* `train_dataset_path` - path to train mds dataset
* `val_dataset_path` - path to test mds dataset
* `model_name` - name FiMMIA neural network architecture
* `num_train_epochs` - number of training epochs
* `output_dir` - path to save FiMMIA model
* `optim` - pytorch optimizer name
* `learning_rate` - learning rate
* `max_grad_norm` - max gradient normalization
* `warmup_ratio` - warmup ratio for optimization
* `sigmas_path` - path for dict with normalization parameters
* `sigmas_type` - type of normalization

As the result we get FiMMIA neural network in directory `data/FiMMIA-Video`. Now we can load model and start Membership Inference Attacks against Large Multimodal LLMs.

In [4]:
from fimmia.fimmia_inference import ModelArguments, init_model

In [7]:
model_args = ModelArguments(
    model_name="FiMMIABaseLineModelLossNormSTDV2",
    model_path="data/FiMMIA-Video"
)
model = init_model(model_args)

In [8]:
model

FiMMIABaseLineModelLossNormSTDV2(
  (loss_component): Sequential(
    (0): Linear(in_features=1, out_features=512, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
  )
  (embedding_component): Sequential(
    (0): Linear(in_features=4096, out_features=2048, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=2048, out_features=512, bias=True)
    (4): Dropout(p=0.2, inplace=False)
    (5): ReLU()
  )
  (attack_encoding): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): Dropout(p=0.2, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=64, bias=True)
    (10): Dropout(p=0.2, inplace=False)
    (11): ReLU()
    (12): Linear(in_features=64, o

### Run MIA inference

Our pretrained models available on 🤗 HuggingFace [FiMMIA collection](https://huggingface.co/collections/ai-forever/fimmia). For Video we download the following model:

```bash
git clone https://huggingface.co/ai-forever/FiMMIA-Video
```

For inference FiMMIA model on new data we should **run command**:

```bash
python job_launcher.py --script="fimmia.fimmia_inference" \
  --model_name="FiMMIABaseLineModelLossNormSTDV2" \
  --model_path="FiMMIA-Video" \
  --test_path="data/train_mds" \
  --save_path="data/predictions.csv" \
  --save_metrics_path="data/metrics.csv" \
  --sigmas_path="data/train_sigmas.json" \
  --sigmas_type="std"
```
Here
* `model_name` - name FiMMIA neural network architecture
* `model_path` - path to load FiMMIA model
* `test_path` - path to test dataset
* `save_path` - path to save predictions
* `save_metrics_path` - path to save metrics

The running command will produce file with prdictions for our dataset for each neighbor and metrics file (if in dataset labels 0, 1).

In [17]:
from fimmia.utils.metrics import get_sample_scores, convert_str

In [20]:
preds = pd.read_csv("data/predictions.csv")
preds["score"] = preds["score"].apply(convert_str)

In [11]:
metrics = pd.read_csv("data/metrics.csv")

In [10]:
preds.head()

,ds_name,hash,label,score
0,CommonVideoQA,-2819276857036119732,0,[0. 0.5009828]
1,CommonVideoQA,-2819276857036119732,0,[0.543804 0.08863425]
2,CommonVideoQA,-2819276857036119732,0,[1.0642736 0. ]
3,CommonVideoQA,-2819276857036119732,0,[0.5575552 0.09166689]
4,CommonVideoQA,-2819276857036119732,0,[1.1353273 0. ]


In [12]:
metrics

,method,auroc,fpr95,tpr05,acc,per_neighbors_acc
0,smia,100.0%,0.0%,100.0%,0.985417,0.8783


For getting predictions for each sample in dataset we should aggregate neighbor scores

$$A(t, m) = \frac{1}{K}\sum_{k=1}^{K}f_{FiMMIA}({\Delta \mathcal{L}^k_{norm}}, \Delta e^k)$$

In [23]:
cscores, labels, y_true, y_pred = get_sample_scores(preds)

  0%|          | 2/4264 [00:00<08:59,  7.90it/s]


In [26]:
print(cscores[:5])

[0.2637179046869278, 0.12685959537823996, 0.3438585201899211, 0.2937004665533702, 0.035671127339204155]


If we wonna make a desicion we can $A(t,m) > 0.5$ -> leak.